### The Perceptron

#### Activation Functions

In [9]:
%run common.ipynb

In [10]:
# %load activation_function.py
"""Numerically stable version of sigmoid::

    def sigmoid(x):
        if x >= 0:
            return 1.0 / (1.0 + math.exp(-x))
        y = math.exp(x)
        return y / (1 + y)

"""

import math


def heaviside(x):
    return 0 if x < 0.0 else 1


def step(x, d=0.5):
    return 0 if x < d else 1


def relu(x):
    return max(0.0, x)


def linear(x, r=1.0):
    return r * x


def dx_linear(_, r=1.0):
    return r


def sigmoid(x):
    return 1.0 / (1.0 + math.exp(-x))


def dx_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))


tanh = math.tanh


def dx_tanh(x):
    return 1 - tanh(x) ** 2


#### Perceptron Implementation

In [11]:
# %load perceptron.py
from vector import dotprod


def perceptron(T, s=0.01, epochs=50):
    n = len(T[0])
    w, b = n * (0.0,), 0.0
    for _ in range(epochs):
        done = True
        for x, y in T:
            if dotprod(w, x) + b >= 0:
                z = 1
            else:
                z = 0
            w = tuple(w[i] - s * (z - y) * x[i] for i in range(n))
            b = b - s * (z - y)
            if not y == z:
                done = False
        if done:
            break
    return w, b


In [12]:
def run_perceptron(f, T, s=0.01, epochs=50):
    w, b = perceptron(T, s=s, epochs=epochs)
    p = lambda x: heaviside(dotprod(w, x) + b)
    if all(p(x) == y for x, y in T):
        print(f"{f.__name__}: ok")
    else:
        print(_red(f"{f.__name__}: failed"))
    for x, y in T:
        if p(x) == y:
            print(f"    {f.__name__}{x} = {y}")
        else:
            print(_red(f"    {f.__name__}{x} = {y} ≠ {p(x)}"))


#### Approximation of Boolean Operators

In [13]:
# %load boolean.py

def AND(x, y):
    return int(x and y)


def OR(x, y):
    return int(x or y)


def NAND(x, y):
    return int(not x and not y)


def XOR(x, y):
    return int(x and not y or not x and y)


BOOLEAN_OPERATORS = AND, OR, NAND, XOR
BOOLEAN_OPERATOR_DOMAIN = (0, 0), (0, 1), (1, 0), (1, 1)
BOOLEAN_TESTDATA = {f: [(x, f(*x)) for x in BOOLEAN_OPERATOR_DOMAIN] for f in BOOLEAN_OPERATORS}
SEPARABLE_BOOLEAN_OPERATORS = AND, OR, NAND
NON_SEPARABLE_BOOLEAN_OPERATORS = XOR


In [14]:
s, epochs = 0.01, 50
for f, T in BOOLEAN_TESTDATA.items():
    run_perceptron(f, T, s=s, epochs=epochs)
    

AND: ok
    AND(0, 0) = 0
    AND(0, 1) = 0
    AND(1, 0) = 0
    AND(1, 1) = 1
OR: ok
    OR(0, 0) = 0
    OR(0, 1) = 1
    OR(1, 0) = 1
    OR(1, 1) = 1
NAND: ok
    NAND(0, 0) = 1
    NAND(0, 1) = 0
    NAND(1, 0) = 0
    NAND(1, 1) = 0
XOR: failed
    XOR(0, 0) = 0 ≠ 1
    XOR(0, 1) = 1
    XOR(1, 0) = 1 ≠ 0
    XOR(1, 1) = 0
